In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle

import numpy as np
from model.ensemble import MLPEnsemble
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from dataloader.builder import build_dataset
from model.mlp import MLP
from uncertainty_estimator.masks import build_masks 
from experiment_setup import build_estimator
from analysis.metrics import get_uq_metrics

plt.rcParams['figure.facecolor'] = 'white'

In [3]:
config = {
    'random_seed': 43,
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    'k_folds': 10,
    'verbose': False,
    'use_cache': False,
    'layers': [8, 256, 256, 128, 1],
    'epochs': 5_000,
    'validation_step': 100,
    'acc_percentile': 0.1,
    'patience': 5,
    'dropout_train': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 512,
    'dataset': 'kin8nm',
    'scale': True,
    'l2_reg': 1e-5
}


In [4]:
# Get data
dataset = build_dataset(config['dataset']) 
x_set, y_set = dataset.dataset('train')
x_test, y_test = dataset.dataset('val')
config['layers'][0] = x_set.shape[-1]

In [5]:
# Helper functions
def scale(train, val):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    val = scaler.transform(val)
    return train, val, scaler


def plot_evaluations(model, x, y, x_, y_):
    plt.figure(figsize=(10, 10))
    plt.plot((min(y), max(y)), (min(y), max(y)))
    plt.scatter(model(x).cpu().numpy(), y)
    plt.scatter(model(x_).cpu().numpy(), y_)
    plt.show()

# Evaluate different masks
def evaluate_masks(model, masks, x_val, y_val, y_scaler):
    predictions = model(x_val).cpu().numpy()
    errors = np.abs(predictions - y_val)
    scaled_errors = y_scaler.inverse_transform(predictions) - y_scaler.inverse_transform(y_val)
    rmse = np.sqrt(np.mean(np.square(scaled_errors)))
    results = []

    for name, mask in masks.items():
        estimator = build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])

        for run in range(config['runs']):
            estimations = estimator.estimate(x_val)
            acc, ndcg, nll = get_uq_metrics(estimations, errors, config['acc_percentile'])
            results.append([acc, ndcg, nll, rmse, name])

            if hasattr(mask, 'reset'):
                mask.reset()

    return results


In [6]:
def multiple_kfold(k, data_size, max_iterations):
    kfold = KFold(k)
    for i in range(max_iterations):
        if i % k == 0:
            data_idx = np.random.permutation(data_size)
            idx_generator = kfold.split(data_idx)
        train_idx, val_idx = next(idx_generator)
        yield data_idx[train_idx], data_idx[val_idx]

In [ ]:
mask_results = [] 
masks = build_masks(['vanilla', 'mirror_random', 'decorrelating', 'dpp'], nn_runs=config['nn_runs'])

kfold_iterator = multiple_kfold(config['k_folds'], len(x_set), config['max_runs'])
for m, (train_idx, val_idx) in enumerate(kfold_iterator):
    x_train, y_train = x_set[train_idx], y_set[train_idx]
    x_val, y_val = x_set[val_idx], y_set[val_idx]
    print("Model", m+1)

    if config['scale']:
        x_train, x_val, _ = scale(x_train, x_val)
        y_train, y_val, y_scaler = scale(y_train, y_val)
    else:
        y_scaler = None
        
    train_opts = ['patience', 'dropout_rate', 'epochs', 'batch_size', 'validation_step']
    train_config = {k: config[k] for k in config if k in train_opts}
    
    model = MLP(config['layers'], l2_reg=config['l2_reg'])
    model.fit((x_train, y_train), (x_val, y_val), **train_config)
    if config['verbose']:
        plot_evaluations(model, x_train, y_train, x_val, y_val)
    
    
    results = evaluate_masks(model, masks, x_val, y_val, y_scaler)
    mask_results.extend(results)
    
    model = MLPEnsemble(config['layers'], 10)
    model.fit((x_train, y_train), (x_val, y_val), **train_config)
    predictions = model(x_val).cpu().numpy()
    errors = np.abs(predictions - y_val)
    scaled_errors = y_scaler.inverse_transform(predictions) - y_scaler.inverse_transform(y_val)
    rmse = np.sqrt(np.mean(np.square(scaled_errors)))
    
    uncertainty = model.compute_uncertainty(x_val).cpu().numpy()
    acc, ndcg, nll = get_uq_metrics(uncertainty, errors, config['acc_percentile'])
    mask_results.append([acc, ndcg, nll, rmse, 'ensemble'])
    
    
    uncertainty = model.compute_uncertainty(x_val, dropout_rate=config['dropout_uq'], repeat=config['nn_runs']).cpu().numpy()
    acc, ndcg, nll = get_uq_metrics(uncertainty, errors, config['acc_percentile'])
    mask_results.append([acc, ndcg, nll, rmse, 'drop_ensemble'])

Model 1
Epoch [100/5000], Loss: 0.0462, Validation loss: 0.0841
Epoch [200/5000], Loss: 0.0416, Validation loss: 0.0969
Epoch [300/5000], Loss: 0.0263, Validation loss: 0.0875
Epoch [400/5000], Loss: 0.0188, Validation loss: 0.0830
Epoch [500/5000], Loss: 0.0152, Validation loss: 0.0814
Epoch [600/5000], Loss: 0.0153, Validation loss: 0.0899
Epoch [700/5000], Loss: 0.0150, Validation loss: 0.0960
Epoch [800/5000], Loss: 0.0092, Validation loss: 0.0896
Epoch [900/5000], Loss: 0.0112, Validation loss: 0.0903
Epoch [1000/5000], Loss: 0.0062, Validation loss: 0.0861
Fit [0/10] model:
Fit [1/10] model:
Fit [2/10] model:
Fit [3/10] model:
Fit [4/10] model:
Fit [5/10] model:
Fit [6/10] model:
Fit [7/10] model:
Fit [8/10] model:
Fit [9/10] model:
Model 2
Epoch [100/5000], Loss: 0.0740, Validation loss: 0.0997
Epoch [200/5000], Loss: 0.0436, Validation loss: 0.0960
Epoch [300/5000], Loss: 0.0304, Validation loss: 0.0857
Epoch [400/5000], Loss: 0.0148, Validation loss: 0.0814
Epoch [500/5000], L

In [ ]:
uncertainty = model.compute_uncertainty(x_val, repeat=1).cpu().numpy()
plt.scatter(errors, uncertainty)
uncertainty = model.compute_uncertainty(x_val, repeat=100, dropout_rate=0.5).cpu().numpy()
plt.scatter(errors, uncertainty)

In [ ]:
# # Plot the results
mask_df = pd.DataFrame(mask_results, columns=['acc', 'ndcg', 'nll', 'rmse', 'mask'])

fig = plt.figure(figsize=(16, 16))
plt.suptitle(config['dataset'], fontsize=16)
fig.subplots_adjust(top=0.95)


def boxplot(df, x_label, y_label, i, bottom=0, top=1):
    plt.subplot(2, 2, i)
    plt.xticks(rotation=45)
    if bottom is not None:
        plt.ylim(bottom, top)
    sns.boxplot(data=df, x=x_label, y=y_label)

boxplot(mask_df, 'mask', 'acc', 1, 0, 0.8)
boxplot(mask_df, 'mask', 'ndcg', 2, 0, 0.9)
boxplot(mask_df, 'mask', 'nll', 3, None)
boxplot(mask_df, None, 'rmse', 4, 0, 1.2*mask_df.rmse.max())

In [ ]:
with open(f"data/results_{config['dataset']}.pkl", 'wb') as f:
    pickle.dump(mask_results, f)

In [ ]:
predictions = model(x_val).cpu().numpy()

In [ ]:
import socket
print(socket.gethostname())

In [4]:
class Foo(object):
     def __init__(self, frob, frotz):
          self.frobnicate = frob
          self.frotz = frotz

class Bar(Foo):
     def __init__(self, frob, frizzle):
          # super(Bar, self).__init__(frob, frizzle)
          super().__init__(frob, frizzle)
          self.frotz = 34
          self.frazzle = frizzle


bar = Bar(1,2)
print("frobnicate:", bar.frobnicate)
print("frotz:", bar.frotz)
print("frazzle:", bar.frazzle)

frobnicate: 1
frotz: 34
frazzle: 2
